In [12]:
from mip import Model, maximize,CONTINUOUS, BINARY, xsum
from collections import deque

## Passo a passo

1. Formular o Problema. Python-mip
2. Criar Nó raiz (representa o problema original)
3. Resolver o Nó raiz (com relaxação)
4. Verificar a Solução. (Se a solução inteira é viável. If not, ramificar)
5. Ramificar (Branching)
    - Identificar a variável mais próxima de 0.5.
    No caso $x_j$.
    - Criar 2 nós filhos.
    x_left =0 , x_rigth = 1
6. Resolver os nós filhos.
7. Poda.
8. Atuaizar UB e LB
9. Continuar a busca
9. Critérios de parada (infeasible ou no solution)


In [4]:
class Problem:
    def __init__(self, objective_coeffs, constraints_coeffs, constraints_rhs):
        self.objective_coeffs = objective_coeffs #coef das funcoes objetivo. ex Max 5x2+6x1. a lista seria [
        self.constraints_coeffs = constraints_coeffs #coefs das restricçoes do problema
        self.constraints_rhs = constraints_rhs # lados direitos das restricoes

In [15]:
class Node:
    node_count = 0

    def __init__ (self,problem, solution=None, objective_value=None, is_integer_solution=None, parent=None, parent_side=None):
        self.node_id = Node.node_count
        Node.node_count += 1 #incrementa o id do node
        self.problem = problem
        self.solution = solution # solução parcial (valor das variáveis)
        self.objective_value = objective_value #(valor da função objetivo)
        self.is_integer_solution = is_integer_solution
        self.parent = parent
        self.parent_side = parent_side # Indica se o nó é filho esquerdo ou direito do pai
        self.left_child = None
        self.right_child = None

    def __str__(self):
        if self.parent:
            parent_id =self.parent.node_id 
            return f"Nó {self.node_id} ({self.parent_side} child of {parent_id}), solução = {self.solution}, valor da função objetivo = {self.objective_value}, solução inteira = {self.is_integer_solution}"
        else:
            return f"Nó {self.node_id} (root node)"

In [11]:
#Exemplo de uso da classe Node

root = Node([0, 0], 0, False)
p1 = Node([1, 0], 5, False, root, "left")
p2 = Node([0, 1], 6, False, root, "right")
p3 = Node([1, 1], 11, True, p1, "left")

print(root)
print(p1)
print(p2)
print(p3)

Nó 0 (root node)
Nó 1 (left child of 0), solução = [1, 0], valor da função objetivo = 5, solução inteira = False
Nó 2 (right child of 0), solução = [0, 1], valor da função objetivo = 6, solução inteira = False
Nó 3 (left child of 1), solução = [1, 1], valor da função objetivo = 11, solução inteira = True


In [ ]:
def solve_subproblem(node,problem,branching_var=None):
    model = Model()

    #variaveis tipo x_1 x_2
    variables = [model.add_var(var_type = CONTINUOUS, name=f'x_{i}') for i in range(len(problem.objective_coeffs))]

    #funcao objetivo

    model.objective = maximize(xsum(problem.objective_coeffs[i] * variables[i] for i in range(len(variables))))

    #Restrições
    for constraint_coeffs, rhs in zip(problem.constraints_coeffs, problem.constraint_rhs):
        model += xsum(constraint_coeffs[i] * variables[i] for i in range(len(variables))) <= rhs


    #Adiciona restrição de ramificação xj = 0 ou xj = 1
    if node.parent_side == 'left':
        model += branching_var == 0
    elif node.parent_side == 'right':
        model += branching_var == 1

    # Resolve o modelo
    model.optimize()

    # Verifica se a solução é inteira
    is_integer_solution = all(var.x == int(var.x) for var in variables)

    # Atualização de Nó
    node.solution = {var.name: var.x for var in variables}

    node.objective_value = model.objective_value
    node.is_integer_solution = is_integer_solution

In [ ]:
def identify_branching_variable(node):
    variables = node.solution
    var_to_be_ramified = min(variables, key = lambda x: abs(variables[x]-0.5))
    return var_to_be_ramified

In [ ]:
def create_children_nodes(node):
    problem = node.problem
    branching_var = identify_branching_variable(node)


    # Criação do nó filho esquerdo
    left_child = Node(problem,solution=dict(node.solution), parent=node, parent_side="left")
    #left_child.solution[branching_var] = 0 isso é estranho
    

In [ ]:
def branch_and_bound(problem):

    raiz = Node(problem)

In [17]:
a = Node()
print(a)

Nó 1 (root node)


In [ ]:
def read_data(file_name):
    pass

In [ ]:
# Relembrando fila
from collections import deque

def bfs(graph, starter):
    visited = set() #conjnt para armazenar os nós visitado
    queue = deque([start])

    while queue:
        node = queue.popleft() #remove o primeiro elemento da fila
        if node not in visited:
            print(node)
            visited.add(node)
            queue.extend(graph[node] - visited) #add vizinhos n visitados a fila


In [11]:
from collections import deque

#deque é uma fila de duas pontas

#criando um deque
queue = deque([1,2,3,4])
queue.popleft()

1

In [12]:
queue

deque([2, 3, 4])